In [1]:
# Construct score volume from score maps of each landmark

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
from multiprocess import Pool

Setting environment for Gordon


In [3]:
from conversion import images_to_volume

In [4]:
train_sample_scheme = 1

In [5]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

In [6]:
downscale_factor = 32
voxel_z_size = section_thickness/(xy_pixel_distance_lossless * downscale_factor)

In [11]:
def load_scoremap_worker(sec, stack, label, downscale_factor):
    try:
#         scoremap = DataManager.load_scoremap(stack=stack, section=sec, label=label, downscale_factor=downscale_factor)
        scoremap = DataManager.load_scoremap(stack=stack, section=sec, label=label, 
                                             downscale_factor=downscale_factor, 
                                             suffix='trainSampleScheme_%d'%train_sample_scheme)
        return (sec-1, scoremap)
    except Exception as e:
        pass
    
f = lambda sec: load_scoremap_worker(sec, stack, label, downscale_factor)
    
def load_scoremaps_parallel(sections, stack, label, downscale_factor):
    pool = Pool(4)
    index_scoremap_tuples = pool.map(lambda sec: load_scoremap_worker(sec, stack, label, downscale_factor), 
                                     sections)
    return dict(filter(None, index_scoremap_tuples))

def load_scoremaps_sequential(sections, stack, label, downscale_factor):
    scoremaps = {}
    for sec in sections:
        t = time.time()
        try:
            scoremaps[sec-1] = DataManager.load_scoremap(stack=stack, section=sec, label=label, downscale_factor=downscale_factor)
        except:
            pass
        print time.time() - t
    return scoremaps

In [13]:
for stack in ['MD591']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
#     for label in structures:
    for label in ['7N']:
#     for label in ['Amb']:

        print label
    
        t = time.time()
        scoremaps = load_scoremaps_parallel(stack=stack, sections=range(first_sec, last_sec+1), label=label, downscale_factor=downscale_factor)
#         scoremaps = load_scoremaps_sequential(stack=stack, sections=range(first_sec, last_sec+1), label=label, downscale_factor=downscale_factor)
#         scoremaps = load_scoremaps_parallel(stack=stack, sections=range(200, 300), label=label, downscale_factor=downscale_factor)
#         scoremaps = load_scoremaps_sequential(stack=stack, sections=range(200, 220), label=label, downscale_factor=downscale_factor)
        sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t))
        
#         del scoremaps
        
        score_volume, score_volume_bbox = images_to_volume(images=scoremaps, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=last_sec-1)

        output_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volumes'))
#         score_volume_filepath = os.path.join(output_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s.bp' % \
#                                             dict(stack=stack, ds=downscale_factor, label=label))
        score_volume_filepath = os.path.join(output_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_%(scheme)d.bp' % \
                                            dict(stack=stack, ds=downscale_factor, label=label, scheme=train_sample_scheme))
        bp.pack_ndarray_file(score_volume, score_volume_filepath)
        
        sys.stderr.write('Create score volume: %.2f seconds\n' % (time.time() - t)) # 50 seconds - load scoremap 4 processes

        score_volume_bbox_filepath = os.path.join(output_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_bbox.txt' % \
                                                  dict(stack=stack, ds=downscale_factor, label=label))        
        np.savetxt(score_volume_bbox_filepath, np.array(score_volume_bbox)[None])
    
        del score_volume, scoremaps

7N
116
134
98
152
99
117
135
153
118
100
136
119
154
101
137
120
102
155
138
121
156
103
139
122
104
157
140
123
105
158
141
124
106
142
125
159
107
126
160
143
108
144
110
127
161
128
145
111
162
129
146
112
163
130
113
147
164
131
114
148
165
132
115
149
166
133
170
150
167
188
171
151
172
168
206
189
173
169
207
190
174
224
208
191
175
225
209
192
176
226
210
193
177
211
194
227
178
212
228
195
179
213
196
229
180
197
214
230
181
198
215
231
182
199
232
216
183
200
233
217
184
201
234
185
202
218
235
219
203
186
236
204
220
187
237
221
205
242
238
222
260
243
239
223
261
244
240
278
262
245
241
279
263
246
296
280
247
264
297
248
281
265
298
249
282
266
300
283
250
267
301
284
251
302
268
285
252
303
269
286
253
304
270
287
254
305
271
255
306
288
272
256
289
273
257
290
308
274
258
309
291
275
259
310
292
276
314
311
293
315
277
312
294
316
332
313
295
317
333
350
368
318
334
351
369
335
319
352
370
336
371
320
353
337
354
321
338
355
322
339
356
340
324
357
341
325
358
342
359
326

Load scoremaps: 45.96 seconds
Create score volume: 48.36 seconds


In [ ]:
# plt.imshow(score_volume[:,:,100], cmap=plt.cm.hot)

In [ ]:
# volume_roi = bp.unpack_ndarray_file(volume_dir + '/%(stack)s_scoreVolume_%(label)s.bp' % {'stack': stack, 'label': '5N'})
# plt.imshow(volume_roi[..., 300]);

In [ ]:
# from skimage.transform import rescale, resize

# def load_scoremap_tb(sec):
#     ''' 
#     Get scoremap. 
#     Assume scoremap is stored as a 2D array for scores of the full frame at thumbnail scale.
    
#     Parameters
#     ----------
#     sec : int
#         section number
    
#     Returns
#     -------
#     2d array
#         downsampled full-frame scoremap
#     '''
#     global stack
        
#     img_fn = contourMap_rootdir + '/%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_denseScoremapThumbnail_outerContour.hdf' % \
#                 {'stack': stack, 'sec': sec}

#     contour_prob_map = load_hdf(img_fn)
    
# #     # crop at upper border of ROI box
# #     xmin, ymin, w, h = detect_bbox_lookup[stack]
# #     contour_prob_map[:ymin] = 0

#     scoremap = resize(contour_prob_map, (h_down, w_down))
#     return scoremap